#### hold-out
-전체 데이터를 미리 일정 비율로 나눠서,그 중 일부는 학습용(train), 나머지는 검증용 또는 테스트용(validation/test) 으로 따로 떼어놓고,나중에 다시 섞지 않고 고정된 상태로 쓰는 방식입니다.

In [1]:
# 예제 데이터 생성

import pandas as pd
import numpy as np
train = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/s11_train.csv')
test = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/s11_train.csv')

train_X = train.drop(['grade'], axis=1)
train_y = train['grade']
test_X = test.drop(['grade'], axis=1)
test_y = test['grade']

In [2]:
# hold-out 방법 적용을 위한 train_test_spilt()
# hold-out: 훈련 데이터를 한번 더 분리하여 전체 데이터를 훈련 데이터, 검증 데이터, 테스트 데이터로 나누는 방법
from sklearn.model_selection import train_test_split
train_X_sub, valid_X,train_y_sub, valid_y = train_test_split(train_X, train_y, test_size=0.3, random_state= 1)

print(train_X.shape, train_y_sub.shape, valid_X.shape, valid_y.shape)

(256, 6) (179,) (77, 6) (77,)


In [ ]:
# 선형 회귀모델 생성

from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(train_X_sub, train_y_sub)

LinearRegression()

In [ ]:
# 검증 데이터 RMSE 확인

from sklearn.metrics import mean_squared_error
pred_val = lr.predict(valid_X)
print(mean_squared_error(valid_y, pred_val, squared=False)) # squared=False: 평균제곱'근'오차를 구하기 위한 옵션 기본 값 True이며 이 경우 평균제곱오차(MSE)

3.2548776483216892


#### k-폴드 교차검증 방법
- 홀드아웃 방법은 데이터를 한 번만 분할하여 모형 성능을 검증하는 것보다 여러 번 분할하여 검증하는 것이 더 안정적인 방법
- 훈련 데이터를 임의의 거의 동일한 크기의 그룹으로 여러 번 나눠서 검증하는 방법 (보통 5~10)
    1. fold 중 1 fold는 검증 데이터로 취급 나머지 k-1개의ㅣ fold는 훈련 데이터
    2. k번 반복 매번 다른 그룹의 fold가 검증 데이터로 사용
    3. 총 k개의 추정치가 계산 최종 교차 검증 추정치는 k개 추정치를 평균내서 계산

In [ ]:
from sklearn.model_selection import cross_val_score # k-fold cross validation 적용 위한 cross_val_score() # 디폴트 5

cv_score = cross_val_score(lr, train_X, train_y, scoring= 'neg_root_mean_squared_error') # 회귀 문제에서는 scoring='neg_root_mean_squared_error' 설정
rmse_score = -cv_score # -를 붙여 원래 값으로 복원
mean_rmse_score = np.mean(rmse_score)

In [ ]:
# cv_score은 음수 처리가 되어 있음 cross_val_score은 값이 클수록 좋은 점수라고 가정하기 때문에 RMSE처럼 작을수록 좋은 값은 음수로 리턴


In [7]:
print('폴드별 rmse:', rmse_score)
print('교차검증 rmse:', mean_rmse_score)

폴드별 rmse: [2.83277317 2.74347932 3.20350288 3.37650829 3.14794378]
교차검증 rmse: 3.0608414901995817


In [9]:
from sklearn.model_selection import KFold # KFold로 세부 옵션 조절 가능

cv = KFold(n_splits = 5, shuffle= True, random_state= 0)

from sklearn.model_selection import cross_val_score
cv_score2 = cross_val_score(lr, train_X, train_y, scoring='neg_root_mean_squared_error', cv=cv) # 위 cv를 활용 cv 파라미터
rmse_score2 = -cv_score2
mean_rmse_score2 = np.mean(rmse_score2)

print(mean_rmse_score2)


3.043609474705183


#### 그리드 서치

- 모델의 최적의 초매개변수를 찾고 모델 선응을 개선하는 방법
1. 초매개변수 범위 지정
2. 모델 학습 및 평가
3. 최적의 초매개변수 선택

In [10]:
# 예제
train = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/s11_train.csv')
test = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/s11_train.csv')

train_X = train.drop(['grade'], axis= 1)
train_y = train['grade']

test_X = test.drop(['grade'], axis=1)
test_y = test['grade']

In [11]:
# 랜덤 포레스트 모델 적용
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(random_state=1)

In [12]:
# 랜덤 포레스트 모델 초매개변수 명칭 확인

params = rf.get_params()

for param_name, param_value in params.items():
    print(param_name, param_value)

bootstrap True
ccp_alpha 0.0
criterion squared_error
max_depth None
max_features 1.0
max_leaf_nodes None
max_samples None
min_impurity_decrease 0.0
min_samples_leaf 1
min_samples_split 2
min_weight_fraction_leaf 0.0
n_estimators 100
n_jobs None
oob_score False
random_state 1
verbose 0
warm_start False


In [19]:
# 초매개변수 정의 총 3x3 9개의 초매개변수 조합
param_grid = {'max_depth': [10, 20, 30], # 최대 트리 깊이
              'ccp_alpha': [0.1, 0.3, 0.5]} #비용-복잡도 가지치기 계수

In [20]:
# gridsearchcv() 최적의 초매개변수 탐색 cv=5 설정 모델 학습 평가시 5-폴드 교차검증 적용

rf_search = GridSearchCV(estimator = rf,
                         param_grid = param_grid,
                         cv = 5,
                         scoring = 'neg_root_mean_squared_error')

rf_search.fit(train_X, train_y)

GridSearchCV(cv=5, estimator=RandomForestRegressor(random_state=1),
             param_grid={'ccp_alpha': [0.1, 0.3, 0.5],
                         'max_depth': [10, 20, 30]},
             scoring='neg_root_mean_squared_error')

In [22]:
# 최적의 초매개변수 조합은 .best_params_를 통해 확인
best_params = rf_search.best_params_
print(best_params)

{'ccp_alpha': 0.5, 'max_depth': 10}


In [24]:
# 각 조매개변수 조합별로 5-폴드 교차검증을 적용했을 때, 최적의 RMSE는 .best_score_를 통해 확인

mean_rmse_score3 = -rf_search.best_score_
print('교차검증 RMSE:', mean_rmse_score3)

교차검증 RMSE: 3.028879073471929


In [ ]:
# Hold-out, k-폴드 교차검증 방법 수행 시 주의 사항

train = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/s11_train.csv')
test = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/s11_train.csv')

train_X = train.drop(['grade'], axis=1)
train_y = train['grade']

test_X = test.drop(['grade'], axis = 1)
test_y = test['grade']

from sklearn.model_selection import train_test_split
train_X_sub, valid_X, train_y_sub, valid_y = train_test_split(train_X, train_y, test_size=0.3, random_state=1)

In [25]:
# 표준 정규화 수치형 설명변수에 적용

from sklearn.preprocessing import StandardScaler
stdscaler = StandardScaler()

num_columns = train_X.select_dtypes('number').columns.to_list()
train_X_numeric_scaled = stdscaler.fit_transform(train_X[num_columns])
valid_X_numeric_sclaed = stdscaler.transform(valid_X[num_columns])
test_X_numeric_sclaed = stdscaler.transform(test_X[num_columns])

#### 파이프라인

In [27]:
# 정형 데이터에서 파이프라인 적용

train = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/s11_train.csv')
test = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/s11_test.csv')

train_X = train.drop(['grade'], axis=1)
train_y = train['grade']

test_X = test.drop(['grade'], axis=1)
test_y = test['grade']

# 데이터 전처리: standardscaler 적용 모형은 svr활용 파이프라인 구축

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.svm import SVR

svr_pipe = Pipeline([
    ('preprocess', StandardScaler()),
    ('regressor', SVR())
])

svr_pipe.fit(train_X, train_y)

Pipeline(steps=[('preprocess', StandardScaler()), ('regressor', SVR())])

In [28]:
# pipe에 이름을 붙이기 번거로울 경우 make_pipeline 사용

from sklearn.pipeline import make_pipeline
from sklearn.svm import SVR

svr_pipe2 = make_pipeline(
    StandardScaler(),
    SVR()
)
svr_pipe2.fit(train_X, train_y)

Pipeline(steps=[('standardscaler', StandardScaler()), ('svr', SVR())])

In [29]:
# 5-k fold 교차검증 활용
from sklearn.model_selection import cross_val_score

cv_score4 = cross_val_score(svr_pipe,
                            train_X,
                            train_y,
                            scoring= 'neg_root_mean_squared_error',
                            cv=5)

rmse_score4 = -cv_score4
maen_rmse_score4 = np.mean(rmse_score4)
print(maen_rmse_score4)

3.040520314114216


### 그리드 서치 적용해보기

In [30]:
# SVR() 하이퍼파라미터 명칭 확인

print('SVR 초매개변수:', SVR().get_params())

SVR 초매개변수: {'C': 1.0, 'cache_size': 200, 'coef0': 0.0, 'degree': 3, 'epsilon': 0.1, 'gamma': 'scale', 'kernel': 'rbf', 'max_iter': -1, 'shrinking': True, 'tol': 0.001, 'verbose': False}


In [31]:
SVR_param = {'regressor__C': np.arange(1, 100, 20)}

In [32]:
# GridSearchCV() 활용해서 최적의 초개매변수 찾아보기

from sklearn.model_selection import GridSearchCV
SVR_search = GridSearchCV(estimator=svr_pipe,
                          param_grid= SVR_param,
                          cv=5,
                          scoring= 'neg_root_mean_squared_error')
SVR_search.fit(train_X, train_y)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocess', StandardScaler()),
                                       ('regressor', SVR())]),
             param_grid={'regressor__C': array([ 1, 21, 41, 61, 81])},
             scoring='neg_root_mean_squared_error')

In [33]:
print(SVR_search.best_params_)
print(-SVR_search.best_score_)

{'regressor__C': 1}
3.040520314114216


#### 범주형 변수와 수치형 변수 혼합 데이터에서 파이프라인 적용

In [37]:
train = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/s11_train2.csv')
test = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/s11_test2.csv')

train_X = train.drop(['grade'], axis=1)
train_y = train['grade']

test_X = test.drop(['grade'], axis=1)
test_y = test['grade']

# 범주형 수치형 컬럼 구분
num_columns = train_X.select_dtypes('number').columns.tolist()
cat_columns = train_X.select_dtypes('object').columns.tolist()

In [38]:
# make_pipeline()을 통해 범주형 변수는 원-핫 인코딩, 수치형 변수는 평균 대치법, 표준화를 실행

from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler

cat_preprocess = make_pipeline(
    OneHotEncoder(handle_unknown= 'ignore', sparse_output= False) # sparse_output:희소행렬
)
num_preprocess = make_pipeline(
    SimpleImputer(strategy='mean'),
    StandardScaler()
)

In [39]:
# ColumnTransformer()를 활용하여 각 전처리 단계를 통합

from sklearn.compose import ColumnTransformer
preprocess = ColumnTransformer(
    [('num', num_preprocess, num_columns),
     ('cat', cat_preprocess, cat_columns)]
)

In [40]:
# 데이터 전처리 단계와 모델링 단계를 통합한 파이프라인 생성

from sklearn.pipeline import Pipeline
from sklearn.svm import SVR

full_pipe = Pipeline(
    [
        ("preprocess", preprocess),
        ('regressor', SVR())
    ]
)

In [43]:
# SVR()의 c 파라미터를 튜닝 파라미터로 지정 (모델별칭)__(하이퍼파라미터 명칭) 순으로 지정
SVR_param  = {'regressor__C': np.arange(1, 100, 20)}

In [44]:
# GridSearchCV()를 활용하여 SVR()에 대한 초매개변수 튜닝 진행 5 k- fold

from sklearn.model_selection import GridSearchCV
SVR_search = GridSearchCV(estimator = full_pipe,
                          param_grid = SVR_param,
                          cv=5,
                          scoring= 'neg_root_mean_squared_error')

SVR_search.fit(train_X, train_y)
print(SVR_search.best_params_)
print(-SVR_search.best_score_)

{'regressor__C': 1}
2.9782526839885586


In [45]:
#최종 예측
test_pred = SVR_search.predict(test_X)
test_pred = pd.DataFrame(test_pred, columns = ['pred'])

In [46]:
# 최종 예측 결과 저장
test_pred.to_csv('submission.csv', index=False)